In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# 1 : df = pd.read_csv('/content/drive/MyDrive/데이터마이닝/Team_Project/temp/scaled_final_target.csv',encoding='utf-8')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터마이닝/Team_Project/temp/selected_target1.csv', encoding='utf-8')

In [ ]:
df = df.drop(columns=['Unnamed: 0'])
print(df.head())

   hour  temperature  humidity  windSpeed  cloudCover  windBearing  \
0     5        35.87      0.61       8.29        0.75        285.0   
1     5        35.87      0.61       8.29        0.75        285.0   
2     5        35.87      0.61       8.29        0.75        285.0   
3     5        35.87      0.61       8.29        0.75        285.0   
4     5        35.87      0.61       8.29        0.75        285.0   

   precipIntensity  precipProbability  day_of_week  gen_lag_1  icon_clear-day  \
0              0.0                0.0            4   0.003417           False   
1              0.0                0.0            4   0.003450           False   
2              0.0                0.0            4   0.003417           False   
3              0.0                0.0            4   0.003417           False   
4              0.0                0.0            4   0.003433           False   

   icon_clear-night  icon_cloudy  icon_fog  icon_partly-cloudy-day  \
0             False   

In [ ]:
df.describe()

,hour,temperature,humidity,windSpeed,cloudCover,windBearing,precipIntensity,precipProbability,day_of_week,gen_lag_1,use [kW]
count,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000,503699.000000
mean,11.760440,50.744139,0.664062,6.650846,0.225928,202.332308,0.002599,0.056475,2.963373,0.076261,0.859188
std,6.845837,19.116144,0.194413,3.982699,0.289937,106.532896,0.011259,0.165865,2.182507,0.128445,1.058312
min,0.000000,-12.640000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,35.770000,0.510000,3.660000,0.040000,148.000000,0.000000,0.000000,1.000000,0.003367,0.367842
50%,12.000000,50.330000,0.680000,5.930000,0.120000,208.000000,0.000000,0.000000,2.000000,0.004300,0.562417
75%,18.000000,66.280000,0.840000,8.940000,0.290000,295.000000,0.000000,0.000000,5.000000,0.084000,0.970342
max,23.000000,93.720000,0.980000,22.910000,1.000000,359.000000,0.191000,0.840000,6.000000,0.613883,14.714567


In [ ]:
import pandas as pd
import numpy as np

# q3 (75th Percentile) 기준 이상값으로 분류
threshold = df['use [kW]'].quantile(0.75)
df['target'] = np.where(df['use [kW]'] > threshold, 1, 0)

# 타겟 변수 비율 확인
print(df['target'].value_counts(normalize=True))

target
0    0.75
1    0.25
Name: proportion, dtype: float64


In [ ]:

from sklearn.model_selection import train_test_split

X = df.drop(columns=['target', 'use [kW]'])  # 독립변수
y = df['target']  # 타겟변수

# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)

from imblearn.over_sampling import SMOTE

# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train, y_train)

print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", pd.Series(y_train_SMOTE).value_counts())

(352589, 19) (75555, 19) (75555, 19)
Before SMOTE: target
0    264464
1     88125
Name: count, dtype: int64
After SMOTE: target
0    264464
1    264464
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 정의 (가중치 적용)
logistic_model = LogisticRegression(class_weight='balanced', random_state=42)
logistic_model.fit(X_train_SMOTE, y_train_SMOTE)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [ ]:
# 모델 평가
y_pred = logistic_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# 예측 및 평가
print(classification_report(y_test, y_pred))

# ROC AUC Score 확인
roc_auc = roc_auc_score(y_test, logistic_model.predict_proba(X_test)[:, 1])
print("ROC AUC Score:", roc_auc)

              precision    recall  f1-score   support

           0       0.82      0.59      0.69     56519
           1       0.34      0.63      0.44     19036

    accuracy                           0.60     75555
   macro avg       0.58      0.61      0.56     75555
weighted avg       0.70      0.60      0.63     75555

ROC AUC Score: 0.6433033767054317


In [ ]:
# 회귀 계수 및 절편 추출
b1 = logistic_model.coef_  # 각 독립 변수의 계수 b1
b0 = logistic_model.intercept_  # b0

print("b1:", b1)
print("b0:", b0)

b1: [[ 1.53527220e-03 -2.82145018e-02  4.53115175e-01  1.76601357e-02
  -1.24572469e-01 -1.11275304e-06  2.25597640e-02  1.51079754e-01
  -7.43886898e-02 -1.48460934e+00  5.63506760e-01 -2.90140437e-02
   2.52383844e-02  8.90437139e-02  3.98802561e-02  1.32601229e-01
   1.26517108e-01 -2.04269038e-02 -1.95675322e-01]]
b0: [1.08952919]


In [ ]:
from sklearn.metrics import confusion_matrix

# 예측값 생성 (y_val은 실제 값, logistic_predicted_value는 예측값)
conf_matrix = confusion_matrix(y_val,y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[37227 19564]
 [12537  6227]]


In [ ]:
from sklearn.metrics import recall_score

# 재현율 계산
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

Recall: 0.6253414582895567


In [ ]:
from sklearn.metrics import precision_score

# 정밀도 계산
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

Precision: 0.3394354148845167


In [ ]:
from sklearn.metrics import f1_score

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


F1 Score: 0.4542723004694836


In [ ]:
#from sklearn.metrics import r2_score
from sklearn.metrics import log_loss

# Pseudo R^2
log_likelihood_model = -log_loss(y_val, y_pred, normalize=False)

In [ ]:
print(log_likelihood_model)

-1157037.3174440495


# Lasso

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

# Lasso 회귀 모델 초기화
lasso = Lasso(alpha=1.0)
lasso.fit(X_train_SMOTE, y_train_SMOTE)

# 예측 (연속형 예측값을 0.5 기준으로 이진화)
lasso_predicted_value = (lasso.predict(X_val) >= 0.5).astype(int)

# 평가 지표 계산
lasso_conf_matrix = confusion_matrix(y_val, lasso_predicted_value)
lasso_recall = recall_score(y_val, lasso_predicted_value)
lasso_precision = precision_score(y_val, lasso_predicted_value)
lasso_f1 = f1_score(y_val, lasso_predicted_value)

print("Lasso Model Evaluation:")
print("Confusion Matrix:\n", lasso_conf_matrix)
print("Recall:", lasso_recall)
print("Precision:", lasso_precision)
print("F1 Score:", lasso_f1)


Lasso Model Evaluation:
Confusion Matrix:
 [[32487 24304]
 [ 7208 11556]]
Recall: 0.6158601577488808
Precision: 0.32225320691578363
F1 Score: 0.4231107205623902


In [ ]:
from sklearn.linear_model import Ridge

# Ridge 회귀 모델 초기화
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_SMOTE, y_train_SMOTE)

# 예측 (연속형 예측값을 0.5 기준으로 이진화)
ridge_predicted_value = (ridge.predict(X_val) >= 0.5).astype(int)

# 평가 지표 계산
ridge_conf_matrix = confusion_matrix(y_val, ridge_predicted_value)
ridge_recall = recall_score(y_val, ridge_predicted_value)
ridge_precision = precision_score(y_val, ridge_predicted_value)
ridge_f1 = f1_score(y_val, ridge_predicted_value)

print("\nRidge Model Evaluation:")
print("Confusion Matrix:\n", ridge_conf_matrix)
print("Recall:", ridge_recall)
print("Precision:", ridge_precision)
print("F1 Score:", ridge_f1)



Ridge Model Evaluation:
Confusion Matrix:
 [[35200 21591]
 [ 6836 11928]]
Recall: 0.6356853549349819
Precision: 0.3558578716548823
F1 Score: 0.4562859820591779


In [ ]:
from sklearn.linear_model import ElasticNet

# ElasticNet 회귀 모델 초기화
elasticnet = ElasticNet(alpha=1.0, l1_ratio=0.5)
elasticnet.fit(X_train_SMOTE, y_train_SMOTE)

# 예측 (연속형 예측값을 0.5 기준으로 이진화)
elasticnet_predicted_value = (elasticnet.predict(X_val) >= 0.5).astype(int)

# 평가 지표 계산
elasticnet_conf_matrix = confusion_matrix(y_val, elasticnet_predicted_value)
elasticnet_recall = recall_score(y_val, elasticnet_predicted_value)
elasticnet_precision = precision_score(y_val, elasticnet_predicted_value)
elasticnet_f1 = f1_score(y_val, elasticnet_predicted_value)

print("\nElastic Net Model Evaluation:")
print("Confusion Matrix:\n", elasticnet_conf_matrix)
print("Recall:", elasticnet_recall)
print("Precision:", elasticnet_precision)
print("F1 Score:", elasticnet_f1)



Elastic Net Model Evaluation:
Confusion Matrix:
 [[32695 24096]
 [ 7235 11529]]
Recall: 0.6144212321466638
Precision: 0.3236210526315789
F1 Score: 0.42394601849638713


In [ ]:
# bool 타입 컬럼을 찾아 변환
bool_columns = df.select_dtypes(include=['bool']).columns

# bool 컬럼을 1과 0으로 변환
df[bool_columns] = df[bool_columns].astype(int)

# 변환 결과 확인
print(df.dtypes)

hour                          int64
temperature                 float64
humidity                    float64
windSpeed                   float64
cloudCover                  float64
windBearing                 float64
precipIntensity             float64
precipProbability           float64
day_of_week                   int64
gen_lag_1                   float64
icon_clear-day                int64
icon_clear-night              int64
icon_cloudy                   int64
icon_fog                      int64
icon_partly-cloudy-day        int64
icon_partly-cloudy-night      int64
icon_rain                     int64
icon_snow                     int64
icon_wind                     int64
use [kW]                    float64
target                        int64
dtype: object


In [ ]:

import statsmodels.api as sm

# statsmodels는 절편을 자동으로 추가하지 않으므로, 수동으로 추가
X_train_sm = sm.add_constant(X_train_SMOTE)
X_test_sm = sm.add_constant(X_test)

# 로지스틱 회귀 모델 생성 및 훈련
model = sm.Logit(y_train_SMOTE, X_train_sm)
result = model.fit(maxiter=20)
pred_test = result.predict(X_test_sm)
pred_train = result.predict(X_train_sm)

# 결과 요약 출력
print(result.summary())

         Current function value: 0.648711
         Iterations: 20


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               528928
Model:                          Logit   Df Residuals:                   528908
Method:                           MLE   Df Model:                           19
Date:                Tue, 10 Dec 2024   Pseudo R-squ.:                 0.06411
Time:                        08:13:33   Log-Likelihood:            -3.4312e+05
converged:                      False   LL-Null:                   -3.6662e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       24.6965   1853.515      0.013      0.989   -3608.126    3657.519
hour                         0.0077      0.000     18.209      0.000       0.007       0.